In [2]:
import string
import pandas as pd
import networkx as nx
from scipy.optimize import root, fsolve
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

In [17]:
delta = 60*15  # timestep
dataset = 'InVS15_1week'

time_exposed = 3.7*86400
time_presympt = 1.5*86400
time_sympt = 2.3*86400

rp = 1.    # reduction of beta for presymptomatics
rbeta = 0.5  # reduction of beta for asymptomatics and mildly symptomatics

p_asympt = 0.2
p_mild = 0.72

beta = 1.37e-3
tmax = 100*86400/delta  # max simulation time

ntrials = 1000 # number of runs

In [18]:
g = {}
setnodes = set()
totweight = 0.
f = open('tij_' + dataset + '.dat')
(t0,i,j,w) = map(int,f.readline().split())
f.close()

set_w = set()

G_agg = nx.Graph()

f = open('tij_' + dataset + '.dat')
for line in f:
    (t,i,j,w) = map(int,line.split())
    if t not in g:
        g[t] = nx.Graph()
    g[t].add_edge(i,j,weight = w)
    set_w.add(w)
    totweight += w
    setnodes.add(i)
    setnodes.add(j)
    wa = 0
    if G_agg.has_edge(i,j):
        wa = G_agg.get_edge_data(i,j)['weight']
    G_agg.add_edge(i,j,weight= wa+w)
f.close()

t_last = t
n_nodes = len(setnodes)
duration = (t_last -t0)*delta
ndays = int(duration/86400) + 1
length_data_to_loop = int(ndays*86400/delta)
for t in range(length_data_to_loop):
    if t not in g:
        g[t] = nx.Graph()
    for i in setnodes:
        if i not in g[t]:
            g[t].add_node(i)

In [19]:
transmissionprobas = {}
for w in set_w:
    for r in [1,rbeta,rp]:
        transmissionprobas[(w,r)] = 1.-(1-beta*r)**w

In [20]:
def contagionprocess(g_inst, t, status, setS, setE, setIp, setIa, setIm, setIs, setRam, setRs ,transmissionprobas):
    new_status = {}
    exposed_to_add, ip_to_add, ia_to_add, im_to_add, is_to_add = set(), set(), set(), set(), set()
    ram_to_add, rs_to_add = set(), set()
    
    # transmission
    for i in setIp.union(setIa).union(setIm).union(setIs):
        r = 1
        if i in setIp:
            if status[i][0] == 'Ip_s':
                r = rp
            else:
                r = rbeta
        elif i in setIa or i in setIm:
            r = rbeta
            
        for j in g_inst.neighbors(i):                        # S neighbours can be infected and become E
            if status[j] != 'S':
                continue
            w = g_inst.get_edge_data(i,j)['weight']
            if random.uniform(0,1) < transmissionprobas[(w,r)]:
                duration = int(random.normal(time_exposed,time_exposed/10.)/delta)   # duration in exposed phase
                new_status[j] = ('E', t + duration)
                # extract time in which the node will change state again
                exposed_to_add.add(j)
                
    # Evolution from E to Ip
    for i in setE:
        if t > status[i][1]:
            duration = int(random.normal(time_presympt,time_presympt/10.)/delta)    # time of asymptomaticity
            c = random.uniform(0,1)
            if c < p_asympt:
                new_status[i] = ('Ip_a', t+duration)    # presympt becoming asymp
            elif c < p_asympt + p_mild:
                new_status[i] = ('Ip_m', t+duration)     # presympt becoming mildsympt
            else:
                new_status[i] = ('Ip_s', t+duration)    # presympt becoming severesympt
            ip_to_add.add(i)
    
    # Evolution from Ip to Ia, Is or Im
    for i in setIp:
        if t > status[i][1]:
            duration = int(random.normal(time_sympt,time_sympt/10.)/delta)  # duration of symptoms
            if status[i][0] == 'Ip_a':
                new_status[i] = ('Ia', t+duration)
                ia_to_add.add(i)
            elif status[i][0] == 'Ip_m':
                new_status[i] = ('Im', t+duration)
                im_to_add.add(i)
            else:
                new_status[i] = ('Is', t+duration)
                is_to_add.add(i)             
    
    # Evolution from I (Ia, Im or Is) to R
    for i in setIa.union(setIm):
        if t > status[i][1]:
            new_status[i] = ('R')
            ram_to_add.add(i)
    for i in setIs:
        if t > status[i][1]:
            new_status[i] = ('R')
            rs_to_add.add(i)
            
    #update of status and set of infectious
    for i in new_status:
        status[i] = new_status[i]    
        
    setS = setS - exposed_to_add
    setE = setE.union(exposed_to_add) - ip_to_add
    setIp = setIp.union(ip_to_add) - ia_to_add - im_to_add - is_to_add

    setIa = setIa.union(ia_to_add) - ram_to_add
    setIm  = setIm.union(im_to_add) - ram_to_add
    setIs = setIs.union(is_to_add) - rs_to_add

    setRam = setRam.union(ram_to_add)
    setRs = setRs.union(rs_to_add)
    
    
    return setS, setE, setIp, setIa, setIm, setIs, setRam, setRs

In [21]:
ntrials = 2000
step = 0.01 # step for P(R)
smalloutbreak = 0.1 # separation small/large outbreak

list_S, list_E, list_Ip, list_Iam, list_Is, list_Ram, list_Rs = {}, {}, {}, {}, {}, {}, {}
    
for itrial in range(ntrials):
    status = {}
    setS, setE, setIp, setIa, setIm, setIs = set(), set(), set(), set(), set(), set()
    setRam, setRs  = set(), set()
    flags = {}

    for i in setnodes:
        status[i] = 'S'
        setS.add(i)

    real_current_time = 0
    current_time = 0
    # initial seed
    i0 = random.choice(list(setnodes))
    # choice of nexttime
    duration = int(random.normal(time_exposed,time_exposed/10.)/delta)
    status[i0] = ('E',current_time + duration)
    setS.remove(i0)
    setE.add(i0)

    while len(setS.union(setRam).union(setRs)) < n_nodes and real_current_time < tmax:
        current_time = real_current_time % length_data_to_loop
        
        setS, setE, setIp, setIa, setIm, setIs, setRam, setRs = contagionprocess(g[current_time], real_current_time,
                                                                        status, setS, setE, setIp, setIa, 
                                                                        setIm, setIs, setRam, setRs ,
                                                                        transmissionprobas)

        if real_current_time % 10 == 0:
            if real_current_time not in list_S:
                list_S[real_current_time] = []
                list_E[real_current_time] = []
                list_Ip[real_current_time] = []
                list_Iam[real_current_time] = []
                list_Is[real_current_time] = []
                list_Ram[real_current_time] = []
                list_Rs[real_current_time] = []
                
            list_S[real_current_time].append(len(setS))
            list_E[real_current_time].append(len(setE))
            list_Ip[real_current_time].append(len(setIp))
            list_Iam[real_current_time].append(len(setIa.union(setIm)))
            list_Is[real_current_time].append(len(setIs))
            list_Ram[real_current_time].append(len(setRam))
            list_Rs[real_current_time].append(len(setRs))

            
        real_current_time += 1

    while real_current_time <= tmax:
        if real_current_time % 10 == 0:
            if real_current_time not in list_S:
                list_S[real_current_time] = []
                list_E[real_current_time] = []
                list_Ip[real_current_time] = []
                list_Iam[real_current_time] = []
                list_Is[real_current_time] = []
                list_Ram[real_current_time] = []
                list_Rs[real_current_time] = []
                
            list_S[real_current_time].append(len(setS))
            list_E[real_current_time].append(len(setE))
            list_Ip[real_current_time].append(len(setIp))
            list_Iam[real_current_time].append(len(setIa.union(setIm)))
            list_Is[real_current_time].append(len(setIs))
            list_Ram[real_current_time].append(len(setRam))
            list_Rs[real_current_time].append(len(setRs))
        real_current_time += 1
        

        
n_smalloutbreak = 0

for i in range(ntrials):
    r1 = (list_Ram[tmax][i] + list_Iam[tmax][i])/n_nodes 
    r2 = (list_Rs[tmax][i] + list_Is[tmax][i])/n_nodes
    rtot = r1+r2
    if rtot < smalloutbreak:
            n_smalloutbreak += 1
 

rtot_ave =  (np.average(list_Ip[tmax]) + np.average(list_Iam[tmax]) +
                np.average(list_Is[tmax]) + np.average(list_Ram[tmax]) + np.average(list_Rs[tmax]) ) / n_nodes

    # write fraction of runs affecting less than 10%
wfrac = open('results_nomeasure_rp_%s.dat' %  (rp) ,'w')
wfrac.write(('%s\t%s\n') % (n_smalloutbreak/ntrials, rtot_ave))
wfrac.close()